In [53]:
#imports
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import os 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import TensorBoard


In [28]:
#loading dataset using panda 
#this loads one specific csv file
#dataset = pd.read_csv(r'C:\Users\benja\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data\ETFs\aadr.us.txt')
#dataset.head()
#recursively load all csv files into a list
#sets my working directory so the glob can fetch all the files
os.chdir(r'C:\Users\Admin\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data')
file_list = glob.glob('**/*.*',recursive=True)
dataset = pd.DataFrame()
#merges every dataframe in one big dataframe and check for empty files
for file in file_list:
    if(os.stat(file).st_size!=0):
        temp = pd.read_csv(file,sep=',',header=0)
        dataset = pd.concat([dataset.reset_index(drop=True),temp.reset_index(drop=True)])


Data Preprocessing

In [45]:

#order the dataset by date --> don't know if this is necessary
dataset = dataset.sort_values(by=['Date'])
#droping the date column --> not needed
dataset = dataset.drop(['Date'],axis=1)
#rescale the data to -1-1
scaler = MinMaxScaler(feature_range=(-1,1))
dataset = scaler.fit_transform(dataset)


In [54]:
#use tensorboard to visualize the training process
tensorboard = TensorBoard(log_dir='logs/{}'.format('SPP'))

Splitting data into training, test, validation set

In [46]:
#function for splitting the data since the predicted label is always the stock price for of the "next day"
def split_data(data,look_back):
    x = []
    y = []
    for i in range(len(data)-look_back-1):
        x.append(data[i:(i+look_back),:])
        y.append(data[i+look_back,:])
    return np.array(x),np.array(y)
def split_sets(x,y,x_split):
    x_train = x[:x_split]
    y_train = y[:x_split]
    x_test = x[x_split:]
    y_test = y[x_split:]
    return x_train,y_train,x_test,y_test


In [47]:
x,y = split_data(dataset,2)
#define training split --> 80% training, 10% testing, 10% validation
training_split = 0.8
#get the index where the data should be splitted
x_split = int(np.ceil(len(x)*training_split))
#split the data into training and testing and validation sets
x_train,y_train,x_test,y_test = split_sets(x,y,x_split)
#split the testing data into testing and validation sets with 70% testing and 30% validation
testing_split = 0.7
x_split = int(np.ceil(len(x_test)*testing_split))
x_test,y_test,x_val,y_val = split_sets(x_test,y_test,x_split)


Define model

In [63]:
#defining the model
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
#dropout for regularization --> randomly drops 20% of the neurons
model.add(Dropout(0.2))
model.add(LSTM(15,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(15))
model.add(Dropout(0.2))
#output layer --> fully connected layer
model.add(Dense(y_train.shape[1]))
#compiling the model
model.compile(optimizer='adam',loss='mean_squared_error')
#for description of the model --> none = dimnension is variable
model.summary()



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 2, 50)             11400     
                                                                 
 dropout_15 (Dropout)        (None, 2, 50)             0         
                                                                 
 lstm_16 (LSTM)              (None, 2, 15)             3960      
                                                                 
 dropout_16 (Dropout)        (None, 2, 15)             0         
                                                                 
 lstm_17 (LSTM)              (None, 15)                1860      
                                                                 
 dropout_17 (Dropout)        (None, 15)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                

train the model 

In [64]:
#training the model
#overfitting model on one sample
model.fit(x_train[0].reshape(1,x_train.shape[1],x_train.shape[2]),y_train[0].reshape(1,y_train.shape[1]),epochs=100,batch_size=1,verbose=1,callbacks=[tensorboard])
#model.fit(x_train,y_train,validation_data=(x_val,y_val),batch_size=1,epochs=1,callbacks=[tensorboard])


Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 1.0034
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 0.9931
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9897
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 0.9918
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 0.9758
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.9792
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9715
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9635
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.9677
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9541
Epoch 11/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9466
Epoch 12/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9289
Epoch 13/100
1/1 [==============================] - 0s 10ms

train model 

In [66]:
#train the model on the training set
model.fit(x_train,y_train,validation_data=(x_val,y_val),batch_size=32,epochs=1,callbacks=[tensorboard])

436331/436331 [==============================] - 2631s 6ms/step - loss: 3.4827e-05 - val_loss: 1.2068e-06


In [67]:
#save the model --> current model SPP.h5
model.save('SPP.h5')